In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
import keras
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import applications
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Flatten, Dense, Dropout
from keras.preprocessing import image

import warnings
warnings.filterwarnings('ignore')

In [2]:
train_image='D:/Documents/Capstone/Project -2/Dataset/false_smut_Vs_non-false_smut/train'

In [3]:
train_datagen=ImageDataGenerator(rescale=1./255,
                                 zoom_range=0.2,
                                 shear_range=0.1,
                                vertical_flip=False)

In [4]:
training_set=train_datagen.flow_from_directory(train_image,
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 5776 images belonging to 2 classes.


In [5]:
test_datagen = ImageDataGenerator(rescale = 1./255,
                                 zoom_range=0.2)

test_set=train_datagen.flow_from_directory('D:/Documents/Capstone/Project -2/Dataset/false_smut_Vs_non-false_smut/test',
                                              target_size=(224,224),
                                              class_mode='categorical',
                                              batch_size=5)

Found 524 images belonging to 2 classes.


In [6]:
base_model = tf.keras.applications.InceptionResNetV2(
                     include_top=False,
                     weights='imagenet',
                     input_shape=(224,224,3)
                     )
 
base_model.trainable=False
 
model = tf.keras.Sequential([
        base_model,  
        tf.keras.layers.BatchNormalization(renorm=True),
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])

In [7]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                                                 
 batch_normalization_203 (Ba  (None, 5, 5, 1536)       10752     
 tchNormalization)                                               
                                                                 
 global_average_pooling2d (G  (None, 1536)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               786944    
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                        

In [9]:
early = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=0.001, restore_best_weights=True)

In [10]:
batch_size=32
STEP_SIZE_TRAIN = training_set.n//training_set.batch_size
STEP_SIZE_VALID = test_set.n//test_set.batch_size

# fit model
history = model.fit(training_set, steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=test_set,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=7,
                    callbacks=[early])

Epoch 1/7
1155/1155 [==============================] - 2444s 2s/step - loss: 0.3232 - accuracy: 0.8674 - val_loss: 0.2319 - val_accuracy: 0.9250
Epoch 2/7
1155/1155 [==============================] - 2686s 2s/step - loss: 0.2216 - accuracy: 0.9167 - val_loss: 0.2395 - val_accuracy: 0.9019
Epoch 3/7
1155/1155 [==============================] - 1994s 2s/step - loss: 0.1938 - accuracy: 0.9286 - val_loss: 0.2015 - val_accuracy: 0.9269
Epoch 4/7
1155/1155 [==============================] - 1671s 1s/step - loss: 0.1784 - accuracy: 0.9328 - val_loss: 0.2361 - val_accuracy: 0.9250
Epoch 5/7
1155/1155 [==============================] - 1513s 1s/step - loss: 0.1750 - accuracy: 0.9319 - val_loss: 0.1722 - val_accuracy: 0.9327
Epoch 6/7
1155/1155 [==============================] - 1417s 1s/step - loss: 0.1446 - accuracy: 0.9439 - val_loss: 0.1925 - val_accuracy: 0.9308
Epoch 7/7
1155/1155 [==============================] - 1313s 1s/step - loss: 0.1509 - accuracy: 0.9418 - val_loss: 0.1545 - val_ac